In [3]:
from urllib.request import urlopen
from IPython.display import SVG
import matplotlib.pyplot as plt
from rdkit import Chem
from tqdm import tqdm
import pandas as pd
import xlsxwriter
import argparse
import pickle
import numpy as np
import json
import copy

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import visualizer as visualizer
import utils as utils
import handle_network as hn
import fragmentation_py as fragmentation_py
import library_downloader as library_downloader
import SiteLocator as modSite
import alignment as alignment
from Compound_n import Compound
from ModificationSiteLocator import ModificationSiteLocator

library ="BERKELEY-LAB"
if not os.path.exists( os.path.join("../data/libraries", library)):
    url = "https://gnps-external.ucsd.edu/gnpslibrary/" + library + ".json"
    location = "../data/libraries/" + library + "/"
    library_downloader.download(url, location, 0.5, 0.1)

with open(os.path.join("../data/libraries", library, "data_dict_filtered.pkl"), "rb") as f:
    data_dict_filtered = pickle.load(f)

In [4]:
matches = pickle.load(open("good_matches.pkl", "rb"))

In [13]:
import numpy as np

samples = []
while (len(samples) < 6):
    i = np.random.randint(0, len(matches))
    if matches[i][0] < 0.8:
        continue
    m0, m1 = matches[i][1]
    myDict = {}
    myDict['USI1'] = hn.generate_usi(m0, library)
    myDict['USI2'] = hn.generate_usi(m1, library)
    myDict['Smiles1'] = data_dict_filtered[m0]["Smiles"]
    myDict['Smiles2'] = data_dict_filtered[m1]["Smiles"]
    myDict['score'] = matches[i][0]
    samples.append(myDict)
    # print(hn.generate_usi(m0, library), hn.generate_usi(m1, library), data_dict_filtered[m0]["Smiles"], data_dict_filtered[m1]["Smiles"])



# print samples, add new line for each sample
print(samples[0])


{'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010105317', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010117740', 'Smiles1': 'COc1ccc(-c2coc3cc(OC)c(OC)cc3c2=O)cc1', 'Smiles2': 'COc1ccc(-c2coc3cc(OC)c(OC)cc3c2=O)cc1OC', 'score': array([0.92596108])}


In [12]:
myData = [{'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010126753', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010118619', 'Smiles1': 'O=C(O)[C@@H]1CCC(O)=N1', 'Smiles2': 'CCOC(=O)C1CCC(O)=N1'}, {'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010125637', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010106370', 'Smiles1': 'COc1cc(C(=O)O)ccc1O', 'Smiles2': 'CCOC(=O)c1ccc(O)c(OC)c1'}, {'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010125095', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010120285', 'Smiles1': 'O=C(O)c1cc(O)c(O)c(O)c1', 'Smiles2': 'CCOC(=O)c1cc(O)c(O)c(O)c1'}, {'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010122676', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010108770', 'Smiles1': 'COc1cc(C=O)ccc1O', 'Smiles2': 'COc1cc(C=O)ccc1OC(C)C'}, {'USI1': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010122675', 'USI2': 'mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010104696', 'Smiles1': 'COc1cc(C=O)ccc1O', 'Smiles2': 'CCCOc1ccc(C=O)cc1OC'}]
columns=["USI1", "Smiles1", "USI2", "Smiles2"]
df = pd.DataFrame(myData, columns=["USI1", "Smiles1", "USI2", "Smiles2"])
df

,USI1,Smiles1,USI2,Smiles2
0,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,O=C(O)[C@@H]1CCC(O)=N1,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,CCOC(=O)C1CCC(O)=N1
1,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,COc1cc(C(=O)O)ccc1O,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,CCOC(=O)c1ccc(O)c(OC)c1
2,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,O=C(O)c1cc(O)c(O)c(O)c1,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,CCOC(=O)c1cc(O)c(O)c(O)c1
3,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,COc1cc(C=O)ccc1O,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,COc1cc(C=O)ccc1OC(C)C
4,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,COc1cc(C=O)ccc1O,mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB0001...,CCCOc1ccc(C=O)cc1OC


In [6]:
m0 = matches[0][1][0]
m1 = matches[0][1][1]
print(hn.generate_usi(m0, library), hn.generate_usi(m1, library), data_dict_filtered[m0]["Smiles"], data_dict_filtered[m1]["Smiles"])

mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010126753 mzspec:GNPS:BERKELEY-LAB:accession:CCMSLIB00010118619 O=C(O)[C@@H]1CCC(O)=N1 CCOC(=O)C1CCC(O)=N1
